In [1]:
import sys
sys.path.insert(0, '../scripts')

from contact_matrix import ContactMatrix

In [2]:
contact_matrix = ContactMatrix()
contact_matrix.get_contact()

,0-5,5-10,10-15,15-20,20-25,25-30,30-35,35-40,40-45,45-50,50-55,55-60,60-65,65-70,70-75,75-80
Ages,,,,,,,,,,,,,,,,
0-5,2.867183,1.851837,1.048048,0.641159,0.950843,1.286615,1.301781,1.011407,0.610805,0.382344,0.385729,0.309667,0.188916,0.120919,0.061834,0.025602
5-10,1.690835,10.503204,2.597033,0.783573,0.447586,0.981403,1.279940,1.179934,0.903503,0.451263,0.289562,0.245045,0.187862,0.109177,0.051657,0.027126
10-15,0.723217,3.871082,13.668103,1.784378,0.766617,0.600012,0.821860,1.050018,1.029627,0.626834,0.339521,0.169225,0.103064,0.092125,0.062625,0.035296
15-20,0.369951,0.953958,5.172804,14.808079,2.398467,1.121481,0.789342,1.018908,1.067278,0.913928,0.489268,0.215203,0.106726,0.051327,0.029679,0.014097
20-25,0.582560,0.482292,0.695636,4.228935,4.970096,2.306653,1.352699,1.095772,0.936214,0.981561,0.676973,0.370576,0.148025,0.037350,0.035045,0.020850
25-30,1.016575,0.581582,0.350732,1.265664,2.694451,3.141959,1.860041,1.348412,1.094307,0.896645,0.837267,0.439025,0.202349,0.047806,0.017655,0.009736
30-35,0.952296,1.440396,1.086653,0.665388,1.225132,1.893141,2.266711,1.739970,1.280567,0.947469,0.726542,0.498686,0.250562,0.063774,0.031521,0.016289
35-40,0.832938,1.419829,1.269361,0.922484,0.790436,1.326250,1.650751,2.097374,1.730113,1.069539,0.761361,0.405414,0.247461,0.107590,0.054308,0.014876
40-45,0.519673,0.967533,1.241757,1.352914,0.957478,1.115342,1.422365,1.589809,1.864164,1.345732,0.927305,0.343352,0.195533,0.080981,0.045307,0.021007
